# Demo the Liquidator Agent Logic

This notebook:

1. Fetches the latest crvusd contract data into Python objects using `crvusdsim`.
2. Fetches prices and `ExternalMarket`s from the `baseline` scenario.
3. Instantiates a liquidator agent.
4. Checks if there are users to liquidate in the `Controller` and liquidates them.

In [1]:
from crvusdsim.pool import get
from src.agents.liquidator import Liquidator
from src.sim.scenario import Scenario

%load_ext autoreload
%autoreload 2

In [2]:
liquidator = Liquidator()

ETH_POOL = "weth"
(
    pool,
    controller,
    collateral_token,
    stablecoin,
    aggregator,
    stableswap_pools,
    peg_keepers,
    policy,
    factory,
) = get(ETH_POOL, bands_data="controller")

In [18]:
# Generate markets and prices
config = "../src/configs/scenarios/baseline.json"
scenario = Scenario(config)
markets = scenario.generate_markets()
prices = scenario.generate_pricepaths("../" + scenario.price_config)
scenario.update_market_prices(prices[-1])

# Set liquidation paths
liquidator.set_paths(controller, stableswap_pools, markets)

[INFO][23:16:20][root]-545001: Reading price config from ../src/configs/scenarios/baseline.json.
[INFO][23:16:20][root]-545001: Fetching 1inch quotes.
[INFO][23:16:26][root]-545001: We have 436320 quotes.
[INFO][23:16:26][root]-545001: Fitting external markets against 1inch quotes.
[INFO][23:16:26][root]-545001: Reading price config from ../src/configs/prices/1h_1696108632_1701289032.json.


In [19]:
# Artificially inflate collateral price
collateral = controller.COLLATERAL_TOKEN.address
sample = prices[-1]
_prices = sample._prices
assert collateral in _prices.keys()
_prices[collateral] *= 2
sample.update(_prices)
scenario.update_market_prices(sample)

In [20]:
profits, underwater_debt = liquidator.perform_liquidations(controller)

[INFO][23:16:26][root]-545001: There are 15 users to liquidate.
[INFO][23:16:26][root]-545001: Liquidating user 0xf11f0add0e8e4ee208104d8264fcf1b69c4ceafc with expected profit: 27066.520916.
[INFO][23:16:26][root]-545001: Executing trade Swap(pool=<SimCurveStableSwapPool address=0x4dece678ceceb27446b35c672dc7d61f30bad69e chain=mainnet>, i=0, j=1, amt=28931244833).
[INFO][23:16:26][root]-545001: Executing trade Liquidation(controller=<crvusdsim.pool.sim_interface.sim_controller.SimController object at 0x7fa6bcbd18b0>, position=<crvusdsim.pool.crvusd.controller.Position object at 0x7fa6b700f210>, amt=29004879805159848280064, frac=1000000000000000000, i=0, j=1).
[INFO][23:16:26][root]-545001: Executing trade Swap(pool=External Market (USDC, WETH), i=1, j=0, amt=13519858676061470720).
[INFO][23:16:26][root]-545001: Liquidated user 0xf11f0add0e8e4ee208104d8264fcf1b69c4ceafc with profit: 27066.520916.
[INFO][23:16:26][root]-545001: Liquidating user 0xad34c2d6d0e979d6452caf890719038ed27aa0f3 